<img src="../../../images/banners/pandas-cropped.jpeg" width="600"/>

<a class="anchor" id="intro_to_data_structures"></a>
# <img src="../../../images/logos/pandas.png" width="23"/> Series

## <img src="../../../images/logos/toc.png" width="20"/> Table of Contents 
* [Series is ndarray-like](#series_is_ndarray-like)
* [Series is dict-like](#series_is_dict-like)
* [Vectorized operations and label alignment with Series](#vectorized_operations_and_label_alignment_with_series)
* [Name attribute](#name_attribute)

---

In [2]:
import pandas as pd
import numpy as np

[`Series`](../reference/api/pandas.Series.html#pandas.Series "pandas.Series") is a one-dimensional labeled array capable of holding any data
type (integers, strings, floating point numbers, Python objects, etc.). The axis
labels are collectively referred to as the **index**. The basic method to create a Series is to call:

```python
s = pd.Series(data, index=index)
```

Here, `data` can be many different things:
- a Python dict
- an ndarray
- a scalar value (like 5)

The passed **index** is a list of axis labels. Thus, this separates into a few
cases depending on what **data is**:

**From ndarray**

If `data` is an ndarray, **index** must be the same length as **data**. If no
index is passed, one will be created having values `[0, ..., len(data) - 1]`.

In [4]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])

In [5]:
s

a    0.483038
b    0.833659
c   -1.601198
d    0.994983
e   -0.593743
dtype: float64

In [6]:
s.index

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [7]:
pd.Series(np.random.randn(5))

0    1.363497
1    1.016882
2   -1.056964
3    1.062578
4    0.455391
dtype: float64

> **Note:**
> pandas supports non-unique index values. If an operation
that does not support duplicate index values is attempted, an exception
will be raised at that time. The reason for being lazy is nearly all performance-based
(there are many instances in computations, like parts of GroupBy, where the index
is not used).

**From dict**

Series can be instantiated from dicts:

In [3]:
d = {"b": 1, "a": 0, "c": 2}
pd.Series(d)

b    1
a    0
c    2
dtype: int64

> **Note:**
> When the data is a dict, and an index is not passed, the `Series` index
will be ordered by the dict’s insertion order, if you’re using Python
version >= 3.6 and pandas version >= 0.23.
>
>If you’re using Python < 3.6 or pandas < 0.23, and an index is not passed,
the `Series` index will be the lexically ordered list of dict keys.

In the example above, if you were on a Python version lower than 3.6 or a
pandas version lower than 0.23, the `Series` would be ordered by the lexical
order of the dict keys (i.e. `['a', 'b', 'c']` rather than `['b', 'a', 'c']`).

If an index is passed, the values in data corresponding to the labels in the
index will be pulled out.

In [8]:
d = {"a": 0.0, "b": 1.0, "c": 2.0}

In [9]:
pd.Series(d)

a    0.0
b    1.0
c    2.0
dtype: float64

In [10]:
pd.Series(d, index=["b", "c", "d", "a"])

b    1.0
c    2.0
d    NaN
a    0.0
dtype: float64

> **Note:**
> `NaN` (not a number) is the standard missing data marker used in pandas.

**From scalar value**

If `data` is a scalar value, an index must be
provided. The value will be repeated to match the length of **index**.

In [5]:
pd.Series(5.0, index=["a", "b", "c", "d", "e"])

a    5.0
b    5.0
c    5.0
d    5.0
e    5.0
dtype: float64

<a class="anchor" id="series_is_ndarray-like"></a>
## Series is ndarray-like

`Series` acts very similarly to a `ndarray`, and is a valid argument to most NumPy functions.
However, operations such as slicing will also slice the index.

In [11]:
s[0]

0.4830379048091899

In [12]:
s[:3]

a    0.483038
b    0.833659
c   -1.601198
dtype: float64

In [13]:
s[s > s.median()]

b    0.833659
d    0.994983
dtype: float64

In [14]:
s[[4, 3, 1]]

e   -0.593743
d    0.994983
b    0.833659
dtype: float64

In [15]:
np.exp(s)

a    1.620991
b    2.301724
c    0.201655
d    2.704677
e    0.552257
dtype: float64

> **Note:**
> We will address array-based indexing like `s[[4, 3, 1]]`
later in indexing section in details.

Like a NumPy array, a pandas Series has a `dtype`.

In [7]:
s.dtype

dtype('float64')

This is often a NumPy dtype. However, pandas and 3rd-party libraries
extend NumPy’s type system in a few places, in which case the dtype would
be an [`ExtensionDtype`](../reference/api/pandas.api.extensions.ExtensionDtype.html#pandas.api.extensions.ExtensionDtype "pandas.api.extensions.ExtensionDtype"). Some examples within
pandas are [Categorical data](https://pandas.pydata.org/docs/user_guide/categorical.html#categorical) and [Nullable integer data type](https://pandas.pydata.org/docs/user_guide/integer_na.html#integer-na). See dtypes will be covered in details in later sections.

If you need the actual array backing a `Series`, use [`Series.array`](../reference/api/pandas.Series.array.html#pandas.Series.array "pandas.Series.array").

In [8]:
s.array

<PandasArray>
[ 1.7164320240646531, -1.9708228568913284, -1.8717932169742053,
  1.6830788830594439,  0.5051600829339498]
Length: 5, dtype: float64

Accessing the array can be useful when you need to do some operation without the
index (to disable [automatic alignment](https://pandas.pydata.org/docs/user_guide/dsintro.html#dsintro-alignment), for example which will be covered later).

[`Series.array`](../reference/api/pandas.Series.array.html#pandas.Series.array "pandas.Series.array") will always be an [`ExtensionArray`](../reference/api/pandas.api.extensions.ExtensionArray.html#pandas.api.extensions.ExtensionArray "pandas.api.extensions.ExtensionArray").
Briefly, an ExtensionArray is a thin wrapper around one or more *concrete* arrays like a
[`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray "(in NumPy v1.23)"). pandas knows how to take an `ExtensionArray` and
store it in a `Series` or a column of a `DataFrame`.
See [dtypes](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) for more.

While Series is ndarray-like, if you need an *actual* ndarray, then use
[`Series.to_numpy()`](../reference/api/pandas.Series.to_numpy.html#pandas.Series.to_numpy "pandas.Series.to_numpy").

In [9]:
s.to_numpy()

array([ 1.71643202, -1.97082286, -1.87179322,  1.68307888,  0.50516008])

Even if the Series is backed by a [`ExtensionArray`](https://pandas.pydata.org/docs/reference/api/pandas.api.extensions.ExtensionArray.html#pandas.api.extensions.ExtensionArray"),
[`Series.to_numpy()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_numpy.html#pandas.Series.to_numpy") will return a NumPy ndarray.

<a class="anchor" id="series_is_dict-like"></a>
## Series is dict-like

A Series is like a fixed-size dict in that you can get and set values by index
label:

In [16]:
s["a"]

0.4830379048091899

In [17]:
s["e"] = 12.0

In [18]:
s

a     0.483038
b     0.833659
c    -1.601198
d     0.994983
e    12.000000
dtype: float64

In [19]:
"e" in s

True

In [20]:
"f" in s

False

If a label is not contained, an exception is raised:

Using the `get` method, a missing label will return None or specified default:

In [21]:
s.get("f")

In [22]:
s.get("f", np.nan)

nan

<a class="anchor" id="vectorized_operations_and_label_alignment_with_series"></a>
## Vectorized operations and label alignment with Series

When working with raw NumPy arrays, looping through value-by-value is usually
not necessary. The same is true when working with Series in pandas.
Series can also be passed into most NumPy methods expecting an ndarray.

In [23]:
s + s

a     0.966076
b     1.667317
c    -3.202396
d     1.989965
e    24.000000
dtype: float64

In [24]:
s * 2

a     0.966076
b     1.667317
c    -3.202396
d     1.989965
e    24.000000
dtype: float64

In [25]:
np.exp(s)

a         1.620991
b         2.301724
c         0.201655
d         2.704677
e    162754.791419
dtype: float64

A key difference between Series and ndarray is that operations between Series
automatically align the data based on label. Thus, you can write computations
without giving consideration to whether the Series involved have the same
labels.

In [13]:
s[1:] + s[:-1]

a         NaN
b   -3.941646
c   -3.743586
d    3.366158
e         NaN
dtype: float64

The result of an operation between unaligned Series will have the **union** of
the indexes involved. If a label is not found in one Series or the other, the
result will be marked as missing `NaN`. Being able to write code without doing
any explicit data alignment grants immense freedom and flexibility in
interactive data analysis and research. The integrated data alignment features
of the pandas data structures set pandas apart from the majority of related
tools for working with labeled data.

> **Note:**
> In general, we chose to make the default result of operations between
differently indexed objects yield the **union** of the indexes in order to
avoid loss of information. Having an index label, though the data is
missing, is typically important information as part of a computation. You
of course have the option of dropping labels with missing data via the
**dropna** function.

<a class="anchor" id="name_attribute"></a>
## Name attribute

Series can also have a `name` attribute:

In [26]:
s = pd.Series(np.random.randn(5), name="something")

In [27]:
s

0   -0.867508
1    0.188532
2   -1.112716
3    0.999030
4    0.772603
Name: something, dtype: float64

In [28]:
s.name

'something'

The Series `name` will be assigned automatically in many cases, in particular
when taking 1D slices of DataFrame as you will see in DataFrame section later.

You can rename a Series with the [`pandas.Series.rename()`](../reference/api/pandas.Series.rename.html#pandas.Series.rename "pandas.Series.rename") method.

In [29]:
s2 = s.rename("different")
s2.name

'different'

Note that `s` and `s2` refer to different objects.

## Conclusion

**In this tutorial, you learned:**

- What is a `Series` data structure
- How to build a Series
- How to apply different operations on a Series

In the next section, we explore DataFrame data structures. Series can only contain single list with index, whereas dataframe can be made of more than one series or we can say that a dataframe is a collection of series that can be used to analyse the data.